In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
import tensorflow_datasets as tfds

In [3]:
svhn_data = tfds.load("svhn_cropped")

In [4]:
svhn_train, svhn_test = svhn_data["train"], svhn_data["test"]

In [5]:
train_ds = svhn_train.batch(32)
test_ds = svhn_test.batch(32)

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for x in train_ds: 
        x['image'] = tf.dtypes.cast(x['image'], tf.float16)
        x['image'] = x['image']/255.0
        # print(x["image"])
        train_step(x["image"], x["label"])

    for x in test_ds:
        x['image'] = tf.dtypes.cast(x['image'], tf.float16)
        x['image'] = x['image']/255.0
        test_step(x["image"], x["label"])

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

W0619 02:16:34.184021 4541464000 base_layer.py:2081] Layer my_model is casting an input tensor from dtype float16 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1, Loss: 1.1561328172683716, Accuracy: 63.73725509643555, Test Loss: 0.9035940766334534, Test Accuracy: 74.23939514160156
Epoch 2, Loss: 0.6296172142028809, Accuracy: 82.05085754394531, Test Loss: 0.824359655380249, Test Accuracy: 76.7593765258789
Epoch 3, Loss: 0.5543760061264038, Accuracy: 84.3400650024414, Test Loss: 0.7970996499061584, Test Accuracy: 77.81576538085938
Epoch 4, Loss: 0.5079022645950317, Accuracy: 85.5467758178711, Test Loss: 0.7560363411903381, Test Accuracy: 79.47142028808594
Epoch 5, Loss: 0.4660820960998535, Accuracy: 86.69478607177734, Test Loss: 0.772743821144104, Test Accuracy: 79.8017807006836
